# Przypisanie sentymentu dla jak największej liczby słów

1. Import danych z projektu NAWL - Nencki Affective Word List z oznaczeniem sentymentu
2. Przypisanie sentymentu w oparciu o podstawowe słowa
3. Wykorzystanie embedingów do przypisania sentymentu na podstawie sąsiadujących słów

In [1]:
import re
import pandas as pd
import numpy as np
import funcs 
import importlib
import unicodecsv as csv
import pickle
import random
import gensim
from gensim import corpora, models
from timeit import default_timer as timer
from datetime import datetime, timedelta
import nltk
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import spacy
from collections import Counter
import pyLDAvis
import pyLDAvis.gensim
from sklearn.model_selection import train_test_split

importlib.reload(funcs)

<module 'funcs' from 'D:\\_projekty\\praca\\funcs.py'>

In [2]:
%time dane = pd.read_csv('00_dane/dane_cleanest.csv',  sep='|', encoding='UTF-8')
%time dane['speech_rawest'] = dane['speech_rawest'].apply(lambda x: re.sub(r'[\W\d]',' ', x))
%time dane['speech_rawest'] = dane['speech_rawest'].apply(lambda x: [word for word in x.split()])

Wall time: 14.4 s
Wall time: 31.5 s
Wall time: 5.4 s


In [33]:
dane.tail()

,nr,date_,speech_rawer,full_name,party_final,speech_rawest,topic_LDA_BOW,topic_LDA_TF,opis_LDA_BOW,opis_LDA_TF
158315,158874,31-08-2011,Sojuszowi Lewicy Demokratycznej bardzo zależy...,Romuald Ajchler,Sojusz Lewicy Demokratycznej,"[sojusz, lewica, demokratyczny, zależeć, ziemi...",26.0,8.0,posiedzenie,posiedzenie
158316,158875,31-08-2011,Panie Przewodniczący Krajowej Rady Sądownictw...,Stanisław Rydzoń,Sojusz Lewicy Demokratycznej,"[przewodniczący, krajowy, rada, sądownictwo, z...",16.0,13.0,wymiar sprawiedliwości,przepisy
158317,158876,31-08-2011,Mam pytanie do pana ministra. Panie ministrze...,Stanisław Rydzoń,Sojusz Lewicy Demokratycznej,"[mieć, pytanie, minister, minister, mieć, doty...",28.0,2.0,handel,inwestycje
158318,158877,31-08-2011,Pani Minister! Bardzo dużo mówi się o szkolni...,Stanisław Stec,Sojusz Lewicy Demokratycznej,"[minister, mówić, szkolnictwo, zawodowy, dosto...",7.0,2.0,NaN,inwestycje
158319,158878,31-08-2011,Szanowna Pani Minister! Bardzo ważnym zadanie...,Tadeusz Tomaszewski,Sojusz Lewicy Demokratycznej,"[szanowny, minister, ważny, zadanie, władza, p...",25.0,11.0,samorządy,rodzina


## 1. Wczytanie listy słów z NAWL

In [4]:
NAWL = pd.read_csv('00_dane/nawl-analysis.csv', encoding='UTF-8')
NAWL.head()

,word,category,mean Happiness,mean Anger,mean Sadness,mean Fear,mean Disgust,distance to H,distance to A,distance to S,distance to F,distance to D,distance to N
0,rozkoszny,H,5.296296,1.407407,1.481481,1.555556,1.370370,2.366185,7.513857,7.449380,7.387380,7.536883,4.835511
1,świetny,H,5.666666,1.370370,1.370370,1.370370,1.222222,1.974388,7.555593,7.567725,7.552752,7.690782,4.933031
2,przyjaciel,H,6.037037,1.444444,1.629630,1.555556,1.037037,1.901643,7.838998,7.691452,7.755814,8.135946,5.409170
3,uśmiech,H,6.333334,1.074074,1.296296,1.259259,1.111111,1.016740,8.100848,7.925011,7.938565,8.076864,5.491653
4,pochwała,H,5.222222,1.518519,1.296296,1.370370,1.222222,2.182792,7.264471,7.469688,7.397619,7.539364,4.667134


In [5]:
print('Lista zawiera {} słów.'.format(len(NAWL)))

Lista zawiera 2902 słów.


Analiza wczytanych danych

In [6]:
NAWL.groupby('category')["word"].count()

category
A      98
D      48
F     163
H     147
N     219
S      64
U    2163
Name: word, dtype: int64

In [7]:
NAWL.describe()

,mean Happiness,mean Anger,mean Sadness,mean Fear,mean Disgust,distance to H,distance to A,distance to S,distance to F,distance to D,distance to N
count,2902.000000,2902.000000,2902.000000,2902.000000,2902.000000,2902.000000,2902.000000,2902.000000,2902.000000,2902.000000,2902.000000
mean,2.943486,2.207620,2.161658,2.447183,1.958691,5.507071,6.744082,6.795276,6.520144,6.956904,4.262446
std,1.354380,0.984966,0.990665,1.033234,0.898510,2.040968,0.644592,0.586096,0.655118,0.667682,1.351084
min,1.000000,1.000000,1.000000,1.000000,1.000000,0.593512,3.906429,4.037939,3.549725,3.527269,0.803628
25%,1.733262,1.461538,1.461538,1.653846,1.333333,3.928527,6.334007,6.440185,6.098742,6.541179,3.328216
50%,2.769231,1.846154,1.777778,2.153846,1.615385,5.299247,6.736401,6.794270,6.527452,6.934837,4.151446
75%,4.037037,2.692308,2.576923,3.000000,2.304843,7.033776,7.175239,7.161664,6.951493,7.385088,5.041912
max,6.740741,5.884615,6.346154,6.153846,5.923077,11.734792,8.957582,8.980930,9.032602,9.284170,9.926695


## 2. Przypisanie sentymentu słów

### 2.1 Budowa słownika wszystkich słów

In [13]:
dict_sp = {}
for sp in dane['speech_rawest']:
    for w in sp:
        if w not in dict_sp.keys():
            dict_sp[w]=0
        dict_sp[w]+=1

In [9]:
print('Liczba unikalnych słów w słowniku wynosi {}.'.format(len(dict_sp)))

Liczba unikalnych słów w słowniku wynosi 152498.


In [18]:
df_sp = pd.DataFrame(dict_sp)


ValueError: If using all scalar values, you must pass an index

In [16]:
df_sp.head()

chcieć        205353
zdanie         46410
powiedzieć    114946
temat          36539
skutek         31434
dtype: int64

### 2.2 Dogranie parametrów